In [50]:
import numpy as np
import math
from dataclasses import dataclass
from typing import List, Dict
import pandas as pd
from scipy.optimize import minimize
from copy import deepcopy


In [51]:

@dataclass
class District:
    id: int #Порядковый номер района, нужно для расстояний и матрицы корреспонденций
    type: str #Тип района определяет распределение квалификации агентов и МПТ(спальный\смешанный\центральный)
    coords: tuple #Координаты района определённые на R^2, 
    residents: Dict[str, int] #Распределение агентов по различным параметрам в районе
    workplaces: Dict[str, int] #Распределение рабочих мест по различным параметрам в районе
    salaries: Dict[str, float] #Распределение зарплат по МПТ в районе

In [52]:
#ОБЩАЯ СТРУКТУРА ДЛЯ АГЕНТОВ


TOTAL_RESIDENTS = 25000 #Общее количество агнетов в модели


@dataclass
class Agent: #Определение класса свойств агента
    id: int #Порядковый номер агента, техническое дополнение
    district_id: int  #Номер района проживания агента, нужно для матрицы корреспонденций и вычисления расстояния
    skill: int #квалификация агента
    specialization: int #Отрасль работы, которую агент предпочитает
    salary_exp: float #Зарплатные ожидания агента
    alpha: List[float] #Коэффициенты значимости различных параметров агента

In [53]:
#ОБЩАЯ СТРУКТУРА ДЛЯ МПТ


@dataclass
class Workplace:
    id: int #Номер
    district_id: int #Номер района
    skill: str #Требуемый уровень квал
    specialization: int #Индустрия
    salary: float #Уровень зп
    occupied: bool = False #Занято ли

In [54]:
def initialize_districts() -> List[District]:
    districts = []
    district_id = 0
    a = 1/6

    # Центральные районы (4)  Живет небольшая часть населения (10%). Из них 10% низкой квалификации, 20% средней и 70% высокой. 
    central_salaries = {'low': round(2*a,4), 'medium': round(4*a,4), 'high': round(6*a,4)} #Зарплаты в центре высокая квалификация = 1, остальные меньше

    #Распределение агентов
    residents_center = {
    'low': math.ceil(TOTAL_RESIDENTS * 0.10 * 0.10 * 0.25),    #Низкая квалификация
    'medium': math.ceil(TOTAL_RESIDENTS * 0.10 * 0.20 * 0.25), #Средняя квалификация
    'high': math.ceil(TOTAL_RESIDENTS * 0.10 * 0.70 * 0.25) #Высокая квалификация
    }

    #Распределение МПТ
    workplaces = {
    'low': math.ceil((TOTAL_RESIDENTS * (0.10 * 0.10 +  0.30 * 0.50 +  0.60 * 0.80)*0.3 )/4), #Низкая квалификация 
    'medium': math.ceil((TOTAL_RESIDENTS * (0.10 * 0.20 +  0.30 * 0.30 +   0.60 * 0.15) * 0.4)/4), #Средняя квалификация
    'high': math.ceil((TOTAL_RESIDENTS * (0.10 * 0.70  +  0.30 * 0.20 +  0.60 * 0.05) *0.5)/4) #Высокая квалификация
    }
    for coord in [(0,0.125), (0,-0.125), (0.125,0), (-0.125,0)]:
        districts.append(District(
            id=district_id, type='central', coords=coord,
            residents=residents_center.copy(),
            workplaces=workplaces.copy(),
            salaries=central_salaries.copy()))
        district_id +=1
# Смешанные районы (8)


    mixed_salaries = {'low': round(1.5*a,4), 'medium': round(3*a,4), 'high': round(4.5*a,4)} #Зарплаты ниже чем в центре

    #Распределение агентов
    #Проживает 30 % населения. Из них 50% низкой квалификации, 30% средней и 20% высокой.
    residents = {
        'low': math.ceil(TOTAL_RESIDENTS * 0.30 * 0.50 *0.125), #Низкая квалификация
        'medium': math.ceil(TOTAL_RESIDENTS * 0.30 * 0.30*0.125), #Средняя квалификация
        'high': math.ceil(TOTAL_RESIDENTS * 0.30 * 0.20*0.125)#Высокая квалификация
    }
    workplaces = {
        'low': math.ceil((TOTAL_RESIDENTS * ((0.10 * 0.10 +  0.30 * 0.50 +  0.60 * 0.80)*0.2 ))/8),  #Низкая квалификация
        'medium': math.ceil((TOTAL_RESIDENTS * (0.10 * 0.20 +  0.30 * 0.30 +   0.60 * 0.15) * 0.5)/8), #Средняя квалификация
        'high': math.ceil((TOTAL_RESIDENTS * (0.10 * 0.70  +  0.30 * 0.20 +  0.60 * 0.05) *0.3)/8) #Высокая квалификация
    }
    for coord in [(0,0.25),(0,-0.25),(0.25,0),(-0.25,0),
                  (0.2,0.15),(-0.2,0.15),(0.2,-0.15),(-0.2,-0.15)]:
        districts.append(District(
            id=district_id, type='mixed', coords=coord,
            residents=residents.copy(),
            workplaces=workplaces.copy(),
            salaries=mixed_salaries.copy()))
        district_id +=1

    # Спальные районы (8)

    #Распределение по квалификации жителей спальных районов: 80% низкой квалификации, 15% средней и 5% высокой. 
    sleep_salaries = {'low':round(a,4), 'medium':round(2*a,4), 'high':round(3*a,4)} #Самые маленькие зарплаты
    residents = {
        'low': math.ceil(TOTAL_RESIDENTS * 0.60 * 0.80*0.125), #Низкая квалификация
        'medium': math.ceil(TOTAL_RESIDENTS * 0.60 * 0.15*0.125), #Средняя квалификация
        'high': math.ceil(TOTAL_RESIDENTS * 0.60 * 0.05*0.125) #Высокая квалификация
    }
    workplaces = {
        'low': math.ceil((TOTAL_RESIDENTS * ((0.10 * 0.10 +  0.30 * 0.50 +  0.60 * 0.80)*0.5 ))/8),  #Низкая квалификация
        'medium': math.ceil((TOTAL_RESIDENTS * (0.10 * 0.20 +  0.30 * 0.30 +   0.60 * 0.15) * 0.1)/8), #Средняя квалификация
        'high': math.ceil((TOTAL_RESIDENTS * (0.10 * 0.70  +  0.30 * 0.20 +  0.60 * 0.05) *0.2)/8) #Высокая квалификация
    }
    for coord in [(0,0.5),(0,-0.5),(0.5,0),(-0.5,0),
                  (0.3,0.4),(0.3,-0.4),(-0.3,0.4),(-0.3,-0.4)]:
        districts.append(District(
            id=district_id, type='sleep', coords=coord,
            residents=residents.copy(),
            workplaces=workplaces.copy(),
            salaries=sleep_salaries.copy()))
        district_id +=1
    
    return districts

In [55]:

def initialize_workplaces(districts: List[District]) -> List[Workplace]:
    workplaces = []
    wp_id = 0
    for d in districts:
        for skill in ['low', 'medium', 'high']:
            for _ in range(d.workplaces[skill]):
                workplaces.append(Workplace(
                    id=wp_id, district_id=d.id, skill=skill,
                    specialization=(wp_id % 5) + 1,
                    salary=d.salaries[skill]
                ))
                wp_id +=1
    return workplaces

In [56]:


class ClassicGravityModel:
    def __init__(self, districts):
        self.districts = districts
        self.distance_matrix = self._calculate_distance_matrix()
        self.skill_levels = ['low', 'medium', 'high']
        self.skill_index = {s: i for i, s in enumerate(self.skill_levels)}



    def _calculate_distance_matrix(self):
        return np.array([[math.hypot(d1.coords[0]-d2.coords[0], d1.coords[1]-d2.coords[1]) 
                       for d2 in self.districts] 
                      for d1 in self.districts])
    


    def run(self, beta):
        matrix = np.zeros((len(self.districts), len(self.districts)))
        
        for skill in self.skill_levels:
            for i, src in enumerate(self.districts):
                supply = max(src.residents[skill] - src.workplaces[skill], 0)
                if supply <= 0: continue
                
                total = sum(max(d.workplaces[skill] - d.residents[skill], 0) * 
                          math.exp(-beta * self.distance_matrix[i][j]) 
                          for j, d in enumerate(self.districts))
                
                for j, dst in enumerate(self.districts):
                    demand = max(dst.workplaces[skill] - dst.residents[skill], 0)
                    if demand <= 0: continue
                    
                    weight = (supply * demand * math.exp(-beta * self.distance_matrix[i][j])) / total
                    matrix[i][j] += weight
        
        return matrix

# Оптимизация
def optimize_beta(our_matrix, districts):
    def loss(beta):
        classic = ClassicGravityModel(districts).run(beta[0])
        return np.sum((our_matrix - classic)**2)
    
    res = minimize(loss, [0.1], method='Nelder-Mead')
    return res.x[0]

# Визуализация
def print_matrix(matrix, districts):
    df = pd.DataFrame(matrix,
                     index=[f"{d.type}-{d.id}" for d in districts],
                     columns=[f"{d.type}-{d.id}" for d in districts])
    print(df.round(2))

# Основная программа
if __name__ == "__main__":
    # Инициализация
    districts = initialize_districts()
    workplaces = initialize_workplaces(districts)
    
    # Создаем копии рабочих мест
    workplaces_pref = deepcopy(workplaces)
    workplaces_our = deepcopy(workplaces)
    
    # Запуск моделей
    classic_model = ClassicGravityModel(districts)
    

    

In [57]:
classic_matrix = classic_model.run(3)
print("\nКлассическая гравитационная модель:")
print_matrix(classic_matrix, districts)


Классическая гравитационная модель:
           central-0  central-1  central-2  central-3  mixed-4  mixed-5  \
central-0       0.00       0.00       0.00       0.00     0.00     0.00   
central-1       0.00       0.00       0.00       0.00     0.00     0.00   
central-2       0.00       0.00       0.00       0.00     0.00     0.00   
central-3       0.00       0.00       0.00       0.00     0.00     0.00   
mixed-4        37.94      17.92      23.86      23.86     0.00     0.00   
mixed-5        17.92      37.94      23.86      23.86     0.00     0.00   
mixed-6        23.90      23.90      38.00      17.95     0.00     0.00   
mixed-7        23.90      23.90      17.95      38.00     0.00     0.00   
mixed-8        30.56      20.17      33.83      19.12     0.00     0.00   
mixed-9        30.56      20.17      19.12      33.83     0.00     0.00   
mixed-10       20.17      30.56      33.83      19.12     0.00     0.00   
mixed-11       20.17      30.56      19.12      33.83     0.00 

In [58]:
for i in range(len(classic_matrix)):
    classic_matrix[i][i] = districts[i].residents['low'] + \
        districts[i].residents['medium'] + \
        districts[i].residents['high'] - classic_matrix[i].sum()

In [59]:
for i in range(len(classic_matrix)):
    for j in range(len(classic_matrix)):
        classic_matrix[i][j] = int(np.floor(classic_matrix[i][j]))

In [60]:
classic_matrix

array([[6.260e+02, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00],
       [0.000e+00, 6.260e+02, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 6.260e+02, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, 6.260e+02, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 